In [ ]:
import pandas as pd 
import numpy as np

In [35]:
df_master=pd.read_csv("../Processed Data/prd_master_orders_customers.csv")

Getting Total Spend of Customer Till Date

In [42]:
df_master["total_lifetime_spend"] = (
    df_master.groupby("customer_unique_id")["total_payment_value"]
      .transform("sum")
)

Generating Target Label (Avg Order Value) for Repeated Customers

In [ ]:
repeat_customer_ids = set(
    df_master.loc[df_master["is_repeat_customer"] == 1, "customer_unique_id"]
)
repeat_customers_first_order = df_master[
    (df_master["is_first_order"] == 1) &
    (df_master["customer_unique_id"].isin(repeat_customer_ids))
].copy()

avg_spend_per_cust = df_master.groupby('customer_unique_id')['total_payment_value'].mean().reset_index()
avg_spend_per_cust.columns = ['customer_unique_id', 'target_avg_order_value']

repeat_customers_first_order = repeat_customers_first_order.merge(avg_spend_per_cust, on='customer_unique_id', how='left')

In [54]:
df_order_value=repeat_customers_first_order.drop(columns=['total_lifetime_spend','total_payment_value'])

In [56]:
df_order_value.to_csv("../Processed Data/prd_order_value.csv", index=False)